# Análise de Dados em Python: Manipulação de Tabelas

2023/24 -- João Pedro Neto, DI/FCUL

Uma **tabela** (em inglês, *data frame*) é uma estrutura de dados indexada que armazena uma sequência de registos. Os registos têm todos a mesma estrutura e possuem diferentes tipos de informação. É normal representar os registos em linhas, e os vários tipos de informação em colunas. Esta é uma representação quase universal que vocês conhecem, por exemplo, da forma como o Excel organiza a sua informação.

<center><img src="https://media.geeksforgeeks.org/wp-content/uploads/finallpandas.png" alt="https://www.geeksforgeeks.org/python-pandas-dataframe/" width="650"/></center>

Para trabalhar com tabelas em Python o módulo `pandas` é uma das melhores opções. Vamos aprender a trabalhar com ele.


In [ ]:
import pandas as pd

O pandas pode criar uma tabela a partir de uma lista:

In [ ]:
dados = ["Ana", "Bruno", "Carla", "Diogo", "Eva"]

pd.DataFrame(dados)

Se usarmos um dicionário com chaves associadas a listas do mesmo tamanho, o pandas converte para uma tabela apropriada:

In [ ]:
dados = { 'Nome'    : ["Ana", "Bruno", "Carla", "Diogo", "Eva", "Francisco"],
          'Idade'   : [20, 21, 19, 25, 22, 21],
          'emProgI' : [True, True, False, True, False, True],
          'Cidade'  : ['Lisboa', 'Setubal', 'Coimbra', 'Leiria', 'Lisboa', 'Porto']
        }

df = pd.DataFrame(dados)
df

O pandas permite realizar operações sobre as tabelas.

+ selecionar colunas:

In [ ]:
df[['Nome', 'Cidade']]

+ selecionar linhas de acordo com certos critérios:

In [ ]:
df[ df['Cidade']=='Lisboa' ] # selecionar pessoas de Lisboa

+ iterar pelas várias linhas.

Vamos aproveitar essa capacidade para calcular a média das idades:

In [ ]:
nLinhas, nColunas = df.shape
somaIdades = sum(registo['Idade'] for _,registo in df.iterrows())

print(f'{somaIdades/nLinhas:5.2f}')



---



As tabelas podem ser vistas como bases de dados. É normal neste contexto que cada registo possa ser localizado por um valor único, uma chave ou identificador (abreviamos para id).

Nós estamos habituados a isso: temos um número de cartão de cidadão, um número de carta de condução, número de contribuinte, número de aluno da FCUL, etc.

Vamos adicionar aos dados um campo `id` e defini-lo como chave:

In [ ]:
dados = { 'Id'      : [51321, 50123, 52932, 46431, 49123, 50543],
          'Nome'    : ["Ana", "Bruno", "Carla", "Diogo", "Eva", "Francisco"],
          'Idade'   : [20, 21, 19, 25, 22, 21],
          'emProgI' : [True, True, False, True, False, True],
          'Cidade'  : ['Lisboa', 'Setubal', 'Coimbra', 'Leiria', 'Lisboa', 'Porto']
        }

dfPessoas = pd.DataFrame(dados).set_index('Id') # definir chave
dfPessoas

Desta forma podemos usar a chave para pesquisar o respetivo registo com o uso de uma funcionalidade da tabela designada `loc` (de localização):

In [ ]:
dfPessoas.loc[50543]['Nome']

Podemos igualmente aplicar operações aritméticas a colunas da tabela,

In [ ]:
dfPessoas[ dfPessoas['Idade'].ge(22) ] # quem tem idade >= 22

Entre as operações possíveis, temos `add`, `sub`, `mul`, `div`, `mod`, `ge`, `gt`, `eq`, `ne` (not equal), etc.



---



Para experimentar mais funcionalidades do `pandas` vamos ler um ficheiro com dados de movimentos de dinheiro de várias pessoas (que eu criei para este texto). O ficheiro encontra-se na minha conta GitHub no formato [csv](https://pt.wikipedia.org/wiki/Comma-separated_values).

In [ ]:
url = 'https://raw.githubusercontent.com/jpneto/Prog.I/master/data/movimentos.csv'

df = pd.read_csv(url)

df.head(10)  # mostra as primeiras linhas da tabela

Podemos ter um resumo dos dados com a função `info()`

In [ ]:
df.info()

Observamos que o tipo da coluna `data` não está num formato de datas, sendo provavelmente uma *string*.

Vamos convertê-la para o tipo `datatime`, um tipo Python que representa datas:

In [ ]:
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')
df.info()

In [ ]:
df.head()



---



## Projetar e Restringir Tabelas

O `pandas` permite efetuar várias operações sobre tabelas.

A operação mais simples chamada de **projeção** é selecionar um subconjunto de colunas:


In [ ]:
df2 = df[ ['pessoa', 'categoria'] ]

df2.head()

A operação de selecionar apenas algumas linhas chama-se **restrição** e já a vimos em funcionamento:

In [ ]:
df2 = df[ df['pessoa']=='Carla' ]
df2.head()

Podemos aplicar estas duas operações sequencialmente:

In [ ]:
df2 = df[ ['pessoa', 'categoria'] ][ df['pessoa']=='Carla' ]

df2.head()



---



## Fundir Tabelas

Outra operação é a **fusão** de duas tabelas.

Vamos criar uma segunda tabela contendo apenas os nomes das pessoas da tabela anterior, às quais juntamos as respetivas cidades de origem e os anos de nascimento.



In [ ]:
dfInfo = df[['pessoa']].drop_duplicates()  # remove duplicados
dfInfo['cidade'] = ['Lisboa', 'Porto', 'Setúbal', 'Lisboa', 'Lisboa', 'Leiria']
dfInfo['nascimento'] = [1995, 2001, 1998, 2002, 2000, 1999]
dfInfo

Qual o resultado de fundir a tabela original com esta nova tabela?

In [ ]:
df.merge(dfInfo)



---



## Agregar Tabelas

Uma das funcionalidades possíveis é agregar uma tabela por grupos.

Por exemplo, talvez eu queira investigar os gastos efetuados por cada uma das pessoas em separado.

Estas operações realizam-se através da função `groupBy`.

Neste primeiro exemplo, vamos contar quantos registos de movimentos existem para cada pessoa:

In [ ]:
df.groupby('pessoa').aggregate({'valor':'count'})

Ou mostrar a média dos movimentos por pessoa,

In [ ]:
df.groupby('pessoa').aggregate({'valor':'mean'})

Podemos agregar vários valores na mesma tabela:

In [ ]:
df.groupby('pessoa').aggregate({'valor': ['min', 'mean', 'max'],
                                'data' : ['min', 'max']})

A função `groupBy` devolve uma sequência de tabelas individuais agregadas, neste caso, por pessoa:

In [ ]:
dfPorPessoa = df.groupby("pessoa")

for pessoa, frame in dfPorPessoa:
  print(pessoa)
  print('*'*20)
  print(frame.head(4))

Vamos selecionar, por exemplo, o grupo dos movimentos da Ana:

In [ ]:
dfPorPessoa.get_group("Ana").head()

Agora queremos analisar a soma dos gastos da Ana por cada categoria. Para isso, voltamos a agregar a tabela da Ana por categoria e pedimos para ser calculada a soma de cada grupo:

In [ ]:
dfAna = dfPorPessoa.get_group('Ana')
dfAna.groupby('categoria').sum()

Vamos fazer este cálculo para todas as pessoas. Agregamos primeiro por pessoa e, para cada pessoa, agregamos por categoria. E em cada uma destas agregações calculamos a soma. Armazenamos estas somas num dicionário.

In [ ]:
somas = {} # soma de gastos por pessoa e por categoria

for pessoa, frame1 in df.groupby('pessoa'):              # 1º agrega por pessoa
  somas[pessoa] = []
  categorias = list( frame1.groupby('categoria').groups.keys() )

  for categoria, frame2 in frame1.groupby('categoria'):  # 2º agrega por categoria nessa pessoa
    somas[pessoa].append( round(frame2['valor'].sum(),2) )

print(categorias)
print(somas)

Com o dicionário das somas e a lista das categorias criamos uma nova tabela para ter esta informação bem organizada:

In [ ]:
dfSomas = pd.DataFrame(somas)
dfSomas['categoria'] = categorias
dfSomas.set_index("categoria", drop=True, inplace=True) # a categoria passa a ser a chave
dfSomas

Com esta tabela final podemos criar uma visualização para ser mais fácil compararmos os gastos por pessoa e por categoria:

In [ ]:
dfSomas.plot(kind="bar", figsize=(12,8), rot=30);



---



Podemos agrupar várias linhas e várias colunas,

In [ ]:
df2 = df.groupby(['pessoa','categoria']).aggregate({'valor':['min','max']})
df2.head(13)

Com as colunas achatadas é mais fácil lidar com a tabela,





In [ ]:
df2.columns = ['min valor', 'max valor']
df2 = df2.reset_index() # achatar todas as colunas
df2.head(7)

 O pandas permite criar [pivot tables](https://www.excel-easy.com/data-analysis/pivot-tables.html) (como no Excel), que servem para agrupar ao mesmo tempo duas colunas/grupos de uma tabela

In [ ]:
df = df.merge(dfInfo) # fundir a tabela com as cidades
df.pivot_table(index='categoria', columns='cidade', values='valor', aggfunc='sum')

## Referências

+ [Minimally Sufficient Pandas](https://medium.com/dunder-data/minimally-sufficient-pandas-a8e67f2a2428)

+ [Panda's User Guide](https://pandas.pydata.org/docs/user_guide/index.html)



---



## Interlúdio: Tempos e Datas

Vamos falar um pouco mais sobre o tipo `datetime` para ver algumas das funcionalidades que o Python tem sobre o tempo e as datas.

Gerir datas é um assunto pouco referido nestes tópicos da programação, sendo considerado um tipo mundano de pouco interesse. No entanto, a necessidade de representar datas é muito comum e convém termos um tipo apropriado para o efeito.

Sendo uma informação relativamente simples, existem muitos detalhes subtis (fusos horários, horário de Verão, *leap seconds*, [etc](https://gist.github.com/timvisee/fcda9bbdff88d45cc9061606b4b923ca).) que torna difícil uma abordagem mais exaustiva. Vamos considerar apenas os elementos mais comuns.

In [ ]:
import time

timestamp = time.time()  # os segundos que passaram desde as 00:00:00 UTC de 1 de Janeiro de 1970
timestamp

Este número que indica um *timestamp* do momento da execução do código é útil para um programa mas não para ser visualizado por pessoas. Podemos traduzir para um valor do tipo `datetime`:

In [ ]:
from datetime import datetime

now = datetime.fromtimestamp(timestamp)
now

O módulo permite traduzir um *timestamp* facilmente numa descrição textual. Alguns exemplos:

In [ ]:
print( now.strftime("%d-%m-%Y (%H:%M:%S.%f)") )
print( now.strftime("%H:%M:%S -- %B %d, %Y") )
print( now.strftime("%jº dia do ano %Y") )

Podemos fazer o inverso. Sabendo o formato de data guardada numa *string*, podemos traduzi-la num valor `datetime`.

In [ ]:
from datetime import datetime

datetime.strptime("2022-12-05", "%Y-%m-%d")

Podemos adicionar ou remover tempo a datas:

In [ ]:
from datetime import datetime, timedelta

jumpAWeek = timedelta(days=7)
print(now + jumpAWeek)

jumpAweekMinus10Hours = timedelta(days=7, hours=-10)
print(now + jumpAweekMinus10Hours)

# podemos até somar e subtrair saltos
print(now + jumpAWeek + jumpAweekMinus10Hours)

Podem consultar a [documentação](https://docs.python.org/3/library/datetime.html) para maiores detalhes.



---



## Atividade Opcional: 💰💰

A tabela dos movimentos incluia uma coluna com quantias de dinheiro. Se for necessário implementar um programa sério para lidar com dinheiro, não se deve usar o tipo `float` pelos potenciais erros de arredondamento que podem produzir.

Vamos usar um módulo externo nos próximos exemplos:

In [ ]:
import sys

if 'money' not in sys.modules:
  !pip install money -q

import money

Com este módulo criamos quantias monetárias definindo o valor e a moeda:

In [ ]:
from money import Money

m = Money(amount='20.16', currency='EUR')
m

As operações sobre estes valores são estáveis.

In [ ]:
centimo = Money(amount='0.01', currency='EUR')

(m + centimo) / 2

Calcular o rácio entre duas quantias:

In [ ]:
m2 = Money(amount='5.02', currency='EUR')

m / m2

O módulo tem funcionalidades para formatar quantias:

In [ ]:
m = Money(amount='10000.0', currency='EUR')
print(m.format('en_US', '¤#,##0.00'))
print(m.format('pt_PT', '¤#,##0.00'))
print(m.format('pt_PT', '#,##0.00 ¤¤'))
print(m.format('pt_PT', '#,##0 ¤¤¤'))

m = Money(amount='10000.0', currency='USD')
print(m.format('en_US', '¤#,##0.00')) # em inglês dos USA
print(m.format('pt_PT', '#,##0 ¤¤¤')) # em português de Portugal
print(m.format('es_ES', '#,##0 ¤¤¤')) # em espanhol de Espanha

O módulo também permite fazer conversões de moedas baseado nas conversões correntes. Podem ler a [documentação](https://pypi.org/project/money/) do módulo para mais detalhes.